<a href="https://colab.research.google.com/github/lcipolina/escher/blob/master/ML/Colab-co-mod-gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab-co-mod-gan

Original repo: [zsyzzsoft/co-mod-gan](https://github.com/zsyzzsoft/co-mod-gan)

My fork: [styler00dollar/Colab-co-mod-gan](https://github.com/styler00dollar/Colab-co-mod-gan)

Original Colab

https://github.com/styler00dollar/Colab-co-mod-gan/blob/master/Colab-co-mod-gan.ipynb


In [ ]:
import cv2
import numpy as np
!nvidia-smi

In [ ]:
# @title install

!git clone https://github.com/zsyzzsoft/co-mod-gan
#!git clone styler00dollar/Colab-co-mod-gan
# tensorflow 1.15 to avoid tensorflow.python.framework.errors_impl.NotFoundError
!pip install tensorflow==1.15
%cd /content/co-mod-gan

# Testing

Just place ``/content/input.png`` and ``/content/mask.png``.

In [ ]:
#@title download pretrain
!gdown --id 1dJa3DRWIkx6Ebr8Sc0v1FdvWf6wkd010 # co-mod-gan-places2-050000.pkl
#!gdown --id 1b3XxfAmJ9k2vd73j-3nPMr_lvNMQOFGE # co-mod-gan-ffhq-9-025000.pkl
#!gdown --id 1M2dSxlJnCFNM6LblpB2nQCnaimgwaaKu # co-mod-gan-ffhq-10-025000.pkl

### Upload image and save name
 This is a little bit slower than just drag and drop - the only advantage is the name saving

In [ ]:
from google.colab import files
uploaded = files.upload()
# Get the name of uploaded image:
for key in uploaded.keys():
  uploaded_image = key

In [ ]:
uploaded_image

### Mask drawing code

In [ ]:
from IPython.display import HTML
from google.colab.output import eval_js
from base64 import b64decode

canvas_html = """
<style>
.button {
  background-color: #4CAF50;
  border: none;
  color: white;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  margin: 4px 2px;
  cursor: pointer;
}
</style>
<canvas1 width=%d height=%d>
</canvas1>
<canvas width=%d height=%d>
</canvas>

<button class="button">Finish</button>
<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')

var canvas1 = document.querySelector('canvas1')
var ctx1 = canvas.getContext('2d')


ctx.strokeStyle = 'red';

var img = new Image();
img.src = "data:image/%s;charset=utf-8;base64,%s";
console.log(img)
img.onload = function() {
  ctx1.drawImage(img, 0, 0);
};
img.crossOrigin = 'Anonymous';

ctx.clearRect(0, 0, canvas.width, canvas.height);

ctx.lineWidth = %d
var button = document.querySelector('button')
var mouse = {x: 0, y: 0}

canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}

var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
  }
})
</script>
"""

def draw(imgm, filename='mask.png', w=400, h=200, line_width=1):
  display(HTML(canvas_html % (w, h, w,h, filename.split('.')[-1], imgm, line_width)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)


In [ ]:
width = 6

In [ ]:
import base64, os
from base64 import b64decode
#import cv2
#convert to np array
img = cv2.imread(uploaded_image)
image64 = base64.b64encode(open(uploaded_image, 'rb').read()) #encoding images as text, this is a trick to manipulate the image better.
image64 = image64.decode('utf-8')

pencil_width = (width/100)*img.shape[1]  #sets width of brush
draw(image64, w=img.shape[1], h=img.shape[0], line_width=pencil_width)

#Save the masked image and convert the mask
import matplotlib.pyplot as plt #it uses this garbage to read images and convert to arrays, instead of CV2, wrong choice of library
with_mask = np.array(plt.imread("mask.png")[:,:,:3])
mask = (with_mask[:,:,0]==1)*(with_mask[:,:,1]==0)*(with_mask[:,:,2]==0)
plt.imsave("mask.png",mask, cmap='gray') #save mask

### Resize and convert images if needed

In [ ]:

#@title (optional) read / write / resize file with opencv / invert mask
#@markdown You need masks where ``white = original`` and ``black = inpaint``. Both images are rgb and have the needed dimensions.
import cv2

input_name = uploaded_image

path_image = '/content/co-mod-gan/'  + input_name
path_mask = '/content/co-mod-gan/' + 'mask.png'
image = cv2.imread(path_image)
image = cv2.resize(image, (512,512), cv2.INTER_NEAREST)
cv2.imwrite(path_image, image)

image = cv2.imread(path_mask)
image = 255-image
image = cv2.resize(image, (512,512), cv2.INTER_NEAREST)
cv2.imwrite(path_mask, image)


In [ ]:
# CHECK mask color AND APPLY THIS ONLY IF NEEDED

# ***** mask color inversion ***************************************************************************
# The part of the image that has to be predicted by the model should be in black and the rest in white
# The code above inverts the colors of the mask!
src = '/content/co-mod-gan'
#convert mask
img = cv2.imread(src + 'input.png') # bitwise function needs img as array # use previously saved name    
cv2.imwrite(src + 'mask.png', cv2.bitwise_not(img)) #operations are not done -in place - we need to go back from array to image

In [ ]:
!pip install --upgrade --no-cache-dir gdown
#!gdown --id 1YbSxWraGrQzD7UVOyPF95FCgRz_H03uD # ours
!gdown --id 1dJa3DRWIkx6Ebr8Sc0v1FdvWf6wkd010 #places2


In [ ]:
#Convert Gray to RGB - CoModGans takes RGB channels
#I've just used this online tool
#https://pinetools.com/change-image-brightness

#### ACTUALLY
#The best way to make it work is to paste my image on top of the example img
# otherwise it keeps throwing dimensions error.. as it is missing channels data

In [ ]:
from google.colab import files

In [ ]:
# Run the model
# change name of input and mask files

%cd /content/co-mod-gan
!python run_generator.py -c /content/co-mod-gan/co-mod-gan-places2-050000.pkl -i /content/co-mod-gan/Ecce_Homo.png -m /content/co-mod-gan/mask.png -o /content/output.png

print('DONE')

files.download('/content/output.png') 

In [ ]:
'''
%cd /content/co-mod-gan
!python run_generator.py -c /content/network-snapshot-050060.pkl -i /content/co-mod-gan/imgs/example_image.jpg -m /content/co-mod-gan/imgs/example_mask.jpg -o /content/output.png
'''

# Training

Warnings: Free colab does not have enough RAM. You need Colab Pro with 25GB RAM. Needs ~13GB RAM. Also needs ~10GB VRAM.

Training can also crash, because of:
```  
File "/content/co-mod-gan/metrics/inception_discriminative_score.py", line 30, in _evaluate
    inception = misc.load_pkl('https://drive.google.com/uc?id=1MzTY44rLToO5APn8TZmfR7_ENSe5aZUn') # inception_v3_features.pkl
  File "/content/co-mod-gan/training/misc.py", line 29, in load_pkl
    with open_file_or_url(file_or_url) as file:
  File "/content/co-mod-gan/training/misc.py", line 25, in open_file_or_url
    return dnnlib.util.open_url(file_or_url, cache_dir='.stylegan2-cache')
  File "/content/co-mod-gan/dnnlib/util.py", line 383, in open_url
    raise IOError("Google Drive download quota exceeded -- please try again later")
OSError: Google Drive download quota exceeded -- please try again later
```

In [ ]:
%cd /content/co-mod-gan/dataset_tools
!python create_from_images.py --tfrecord-dir /content/tfrecord_dir --val-image-dir /content/val --train-image-dir /content/train --resolution 512 --num-channels 3 --num-processes 4

In [ ]:
%cd /content/co-mod-gan
# Example: /content/tfrecord_dir/tfrecord_dir-r09.tfrecords
!python run_training.py --data-dir=/content/ --dataset=tfrecord_dir --metrics=ids10k --mirror-augment True --num-gpus=1